# Example Hub and Local Operation Collections

This notebook demonstrates the complete workflow for uploading and downloading operations collections using the 
`HubAnalysisOpManager` and loading local operations via `IT_ANALYSIS_OP_PATHS`. The workflow includes:

1. Setting up local op collection path via IT_ANALYSIS_OP_PATHS
2. Copying the current hub_op_collection folder to /tmp/
3. Uploading operations to HuggingFace Hub as a private repository
4. Downloading the uploaded collection to the default cache
5. Re-importing interpretune to verify both hub and local operations are available
6. Testing the loaded operations
7. Cleaning up downloaded operations and re-importing
8. Verifying only local operations remain available
9. Final cleanup of the local operations collection

```python

**Note**: This example requires HuggingFace Hub authentication and will create a private repository.
```

## Setup and Imports

In [ ]:
import os
import shutil
from pathlib import Path
import sys

# Import interpretune components
import interpretune
from interpretune.utils import rank_zero_warn
from interpretune.analysis.ops.hub_manager import HubAnalysisOpManager
from interpretune.analysis import IT_ANALYSIS_CACHE, IT_ANALYSIS_HUB_CACHE, IT_ANALYSIS_OP_PATHS, IT_MODULES_CACHE
from interpretune.base.components.cli import IT_BASE

example_op_collections_dir = Path(IT_BASE / "notebooks" / "example_op_collections")
example_hub_op_collection_dir = Path(example_op_collections_dir / "hub_op_collection")
example_local_op_collection_dir = Path(example_op_collections_dir / "local_op_collection")
print(f"Interpretune version: {interpretune.version}")
print(f"Current analysis cache location: {IT_ANALYSIS_CACHE}")
print(f"Current modules cache location: {IT_MODULES_CACHE}")
print(f"Current hub cache location: {IT_ANALYSIS_HUB_CACHE}")
print(f"Current IT analysis op paths: {IT_ANALYSIS_OP_PATHS}")
print(f"This notebook's example hub op collection directory: {example_hub_op_collection_dir}")
print(f"This notebook's example local op collection directory: {example_local_op_collection_dir}")

## Step 1: Setup local op collection path

Copy the local_op_collection to /tmp/ and add it to IT_ANALYSIS_OP_PATHS so local operations are loaded.

In [ ]:
# Define source and destination paths for local ops
source_local_op_collection = example_local_op_collection_dir
tmp_local_op_collection = Path("/tmp/local_op_collection")

print(f"Source local op_collection: {source_local_op_collection}")
print(f"Destination: {tmp_local_op_collection}")

# Check if source exists
if not source_local_op_collection.exists():
    raise FileNotFoundError(f"Source local op_collection not found at {source_local_op_collection}")

# Warn if destination already exists
if tmp_local_op_collection.exists():
    rank_zero_warn(f"Destination folder {tmp_local_op_collection} already exists and will be overwritten!")
    shutil.rmtree(tmp_local_op_collection)

# Copy the local op collection folder
shutil.copytree(source_local_op_collection, tmp_local_op_collection)
print(f"✓ Successfully copied local op_collection to {tmp_local_op_collection}")

# Store the original IT_ANALYSIS_OP_PATHS environment variable
original_op_paths_env = os.environ.get('IT_ANALYSIS_OP_PATHS', '')
print(f"Original IT_ANALYSIS_OP_PATHS environment variable: '{original_op_paths_env}'")

# Set the IT_ANALYSIS_OP_PATHS environment variable
# The format is a colon-separated list of paths
new_op_paths = str(tmp_local_op_collection)
if original_op_paths_env:
    new_op_paths = f"{original_op_paths_env}:{new_op_paths}"

os.environ['IT_ANALYSIS_OP_PATHS'] = new_op_paths
print(f"✓ Set IT_ANALYSIS_OP_PATHS environment variable to: '{new_op_paths}'")

# Also update the imported list for consistency (this was the old approach)
if str(tmp_local_op_collection) not in IT_ANALYSIS_OP_PATHS:
    IT_ANALYSIS_OP_PATHS.append(str(tmp_local_op_collection))
    print(f"✓ Also added {tmp_local_op_collection} to imported IT_ANALYSIS_OP_PATHS list")

print(f"\nUpdated IT_ANALYSIS_OP_PATHS list: {IT_ANALYSIS_OP_PATHS}")
print(f"Current IT_ANALYSIS_OP_PATHS env var: '{os.environ.get('IT_ANALYSIS_OP_PATHS', '')}'")

# Verify contents
print("\nContents of copied local op_collection:")
for item in tmp_local_op_collection.iterdir():
    print(f"  - {item.name}")

## Step 2: Copy hub op_collection to /tmp/

Copy the hub op_collection folder to /tmp/ for upload to the hub.

In [ ]:
# Define source and destination paths for hub ops
source_op_collection = example_hub_op_collection_dir
tmp_op_collection = Path("/tmp/hub_op_collection")

print(f"Source hub op_collection: {source_op_collection}")
print(f"Destination: {tmp_op_collection}")

# Check if source exists
if not source_op_collection.exists():
    raise FileNotFoundError(f"Source op_collection not found at {source_op_collection}")

# Warn if destination already exists
if tmp_op_collection.exists():
    rank_zero_warn(f"Destination folder {tmp_op_collection} already exists and will be overwritten!")
    shutil.rmtree(tmp_op_collection)

# Copy the folder
shutil.copytree(source_op_collection, tmp_op_collection)
print(f"✓ Successfully copied hub op_collection to {tmp_op_collection}")

# Verify contents
print("\nContents of copied hub op_collection:")
for item in tmp_op_collection.iterdir():
    print(f"  - {item.name}")

## Step 3: Upload operations to HuggingFace Hub

Upload the hub op_collection to HuggingFace Hub as a private repository named "trivial_op_repo".

In [ ]:
from huggingface_hub import whoami
current_user = whoami()['name']

# Initialize the hub manager
hub_manager = HubAnalysisOpManager()

# Repository configuration
repo_name = "trivial_op_repo"
private = True

print("Uploading op_collection to HuggingFace Hub...")
print(f"Current HF user: {current_user}")
print(f"Repository: {repo_name}")
print(f"Private: {private}")
print(f"Source folder: {tmp_op_collection}")

# Ensure the user is authenticated
repo_id = f'{current_user}/{repo_name}'
try:
    # Upload operations to hub
    # 1. This will create the specified repository if it doesn't exist
    # 2. If the repo exists, it will clean existing operations and upload the new ones in a single commit
    #    - If no files have changed, it will skip the commit and leave the repository unchanged

    upload_result = hub_manager.upload_ops(
        local_dir=tmp_op_collection,
        repo_id=repo_id,
        private=private,
        clean_existing=True,
    )

    print(f"✓ Successfully uploaded operations (if necessary) to {repo_name}")
    print(f"Upload result (new or latest op repo commit sha): {upload_result}")

except Exception as e:
    print(f"❌ Error uploading operations: {e}")
    raise

## Step 4: Download operations to default hub cache

Download the uploaded operations collection to the default `IT_ANALYSIS_HUB_CACHE` location.

In [ ]:
print(f"Downloading operations from {repo_id} to default cache...")
print(f"Cache location: {IT_ANALYSIS_HUB_CACHE}")

# Initialize download_result to None so we can safely check it in cleanup step
download_result = None

try:
    # Download operations from hub to default cache
    download_result = hub_manager.download_ops(
        repo_id=repo_id,
        # cache_dir not specified, will use default IT_ANALYSIS_HUB_CACHE
    )

    print("✓ Successfully downloaded operations to cache")
    print(f"Download result: {download_result}")

    # Check what was downloaded
    cache_path = Path(IT_ANALYSIS_HUB_CACHE)
    if cache_path.exists():
        print("\nContents of hub cache:")
        for item in cache_path.rglob("*"):
            if item.is_file():
                rel_path = item.relative_to(cache_path)
                print(f"  - {rel_path}")

except Exception as e:
    print(f"❌ Error downloading operations: {e}")
    raise

## Step 5: Re-import interpretune and verify hub and local operations

Re-import interpretune to pick up both hub and local operations and verify they are available.

In [ ]:
print("Re-importing interpretune to pick up hub and local operations...")
# Remove interpretune modules from sys.modules to force reimport
modules_to_remove = [name for name in sys.modules.keys() if name.startswith('interpretune')]
for module_name in modules_to_remove:
    del sys.modules[module_name]

# ruff: noqa: E402

# Re-import interpretune
import interpretune as it
from interpretune import DISPATCHER
from interpretune.analysis.ops.base import OpWrapper

print("✓ Interpretune re-imported")

# Get operation definitions and count hub vs local operations
operation_definitions = DISPATCHER.registered_ops
total_ops = len(operation_definitions)

# Get canonical names (resolve aliases to their target operations)
canonical_ops = {}
alias_map = {}
for op_name, op_def in operation_definitions.items():
    canonical_name = op_def.name
    if canonical_name not in canonical_ops:
        canonical_ops[canonical_name] = op_def
        alias_map[canonical_name] = []
    if op_name != canonical_name:
        alias_map[canonical_name].append(op_name)

# Count hub operations (those with dots in their canonical names indicating namespacing)
hub_ops = {name: op_def for name, op_def in canonical_ops.items()
           if '.' in name}

# Count local operations (those without dots but not in the built-in list)
builtin_ops = {'labels_to_ids', 'get_answer_indices', 'get_alive_latents', 'model_forward',
               'model_cache_forward', 'model_ablation', 'model_gradient', 'logit_diffs',
               'logit_diffs_cache', 'sae_correct_acts', 'gradient_attribution', 'ablation_attribution'}

local_ops = {name: op_def for name, op_def in canonical_ops.items()
             if '.' not in name and name not in builtin_ops and op_def.composition is None}

# Count composed operations
composed_ops = {name: op_def for name, op_def in canonical_ops.items()
                if op_def.composition is not None}

print("\n📊 Operation Summary:")
print(f"  Total registered names: {total_ops}")
print(f"  Unique operations: {len(canonical_ops)}")
print(f"  Hub operations: {len(hub_ops)}")
print(f"  Local operations: {len(local_ops)}")
print(f"  Composed operations: {len(composed_ops)}")
print(f"  Built-in operations: {len(builtin_ops)}")

print("\n🌐 Hub operations found:")
for op_name, op_def in hub_ops.items():
    aliases = alias_map.get(op_name, [])
    all_names = [op_name] + aliases
    print(f"  - {op_name} (accessible as: {', '.join(all_names)})")

print("\n🏠 Local operations found:")
for op_name, op_def in local_ops.items():
    aliases = alias_map.get(op_name, [])
    all_names = [op_name] + aliases
    print(f"  - {op_name} (accessible as: {', '.join(all_names)}) - {op_def.description}")

# Test operation instantiation
print("\n🔧 Testing operation instantiation:")
print(f"labels_to_ids op reference type: {type(it.labels_to_ids)}")

print(f"get_answer_indices op reference type: {type(it.get_answer_indices)}")
print(f"trivial_test_op op reference type: {type(it.trivial_test_op)}")

print(f"Get non-direct access attribute of labels_to_ids (description of the underlying AnalysisOp): "
      f"{it.labels_to_ids.description}")
print(f"Type of labels_to_ids now: {type(it.labels_to_ids)}")
print(f"Type of get_answer_indices is still: {type(it.get_answer_indices)} and its instantiated status is "
      f"{it.get_answer_indices._is_instantiated}")
print(f"Non-direct access attribute of get_answer_indices (name of the underlying AnalysisOp): "
      f"{it.get_answer_indices.name}")
print(f"Type of get_answer_indices is now: {type(it.get_answer_indices)}")
print(f"Type of trivial_test_op is: {type(it.trivial_test_op)} and its instantiated status is "
      f"{it.trivial_test_op._is_instantiated}")
try:
    print(f"Non-direct access attribute of trivial_test_op (name of the underlying AnalysisOp): "
          f"{it.trivial_test_op.name}")
    print(f"Type of trivial_test_op is now: {type(it.trivial_test_op)} as it has been successfully instantiated")
except Exception as e:
    print(f"❌ Error instantiating trivial_test_op: {e}")
    if isinstance(it.trivial_test_op, OpWrapper):
        print(f"Type of trivial_test_op is still: {type(it.trivial_test_op)} and its instantiated status is "
              f"{it.trivial_test_op._is_instantiated} likely because of a dynamic import failure.")

try:
    print(f"Non-direct access attribute of trivial_local_test_op (name of the underlying AnalysisOp): "
          f"{it.trivial_local_test_op.name}")
    print(f"Type of trivial_local_test_op is now: {type(it.trivial_local_test_op)} as it has been successfully "
          f"instantiated")
except Exception as e:
    print(f"❌ Error instantiating trivial_local_test_op: {e}")
    if isinstance(it.trivial_local_test_op, OpWrapper):
        print(f"Type of trivial_local_test_op is still: {type(it.trivial_local_test_op)} and its instantiated status "
              f"is {it.trivial_local_test_op._is_instantiated} likely because of a dynamic import failure.")

if hub_ops:
    hub_op_name = next(iter(hub_ops.keys()))
    print(f"{hub_op_name} op reference type: {type(getattr(it, hub_op_name, 'Not found'))}")
if local_ops:
    local_op_name = next(iter(local_ops.keys()))
    print(f"{local_op_name} op reference type: {type(getattr(it, local_op_name, 'Not found'))}")

## Step 6: Test the loaded operations

Test simple hub and local operations both individually executed and as part of a composite operation to ensure loading and execution works correctly.

In [ ]:
print("\n🧪 Testing loaded operations with demo data...")

# Import required components
import torch
from interpretune.analysis.ops.base import AnalysisBatch
from interpretune import trivial_test_op, trivial_local_test_op, composite_trivial_test_op

NUM_BATCHES = 2  # Number of test batches to generate

# Simple generator that creates test analysis_batch objects
def generate_test_batches(num_batches=NUM_BATCHES):
    """Generator that yields test analysis_batch objects with random orig_labels."""
    for i in range(num_batches):
        separate_op_input_batch, pipeline_op_input_batch = AnalysisBatch(), AnalysisBatch()
        orig_labels = torch.randint(0, 5, (4,))
        for batch in (separate_op_input_batch, pipeline_op_input_batch):
            batch.update(orig_labels=orig_labels.clone())
        yield f"Batch {i+1} (random orig_labels)", separate_op_input_batch, pipeline_op_input_batch

VERBOSE_OP_OUTPUTS = False  # Set to True to log operation outputs
def maybe_print(output):
    """Log operation output if logging is enabled."""
    if VERBOSE_OP_OUTPUTS:
        print(output)

# Test the operations
print(f"\n📋 Testing operation pipeline parity of composite vs individual component ops (over {NUM_BATCHES} batches):")
individual_op_output_batches = []
composite_op_output_batches = []
for batch_name, individual_test_batch, composite_test_batch in generate_test_batches():
    print("\nComposite op execution...")
    if VERBOSE_OP_OUTPUTS:
        print(f"\n--- {batch_name} ---")
        print(f"Input batch: {individual_test_batch}")
    composite_output_batch = composite_trivial_test_op(analysis_batch=composite_test_batch)
    maybe_print(f"Composite op output batch: {composite_output_batch}")
    composite_op_output_batches.append(composite_output_batch)
    print("\nRe-running with individual component ops...")
    local_batch_output = trivial_local_test_op(analysis_batch=individual_test_batch)
    maybe_print(f"Local op batch output: {local_batch_output}")
    individual_output_batch = trivial_test_op(analysis_batch=local_batch_output)
    maybe_print(f"Hub output batch: {individual_output_batch}")
    individual_op_output_batches.append(individual_output_batch)

# Compare outputs from individual component op and composite op processing
print("\n🔍 Validating that composite and individual component op outputs are identical...")
all_match = True
for idx, (sep_batch, composite_batch) in enumerate(zip(individual_op_output_batches, composite_op_output_batches)):
    if sep_batch == composite_batch:
        print(f"  ✓ Batch {idx+1}: Outputs match.")
    else:
        print(f"  ❌ Batch {idx+1}: Outputs do NOT match!")
        print(f"    Individual op output: {sep_batch}")
        print(f"    Composite op output: {composite_batch}")
        all_match = False

if all_match:
    print("\n🎉 All batches match: individual and composite operation outputs are identical!")
else:
    print("\n⚠️ Some batches did not match: please check the operation implementations.")


## Step 7: Clean up hub operations and re-import

Delete the downloaded hub operations folder and re-import interpretune to verify only local operations remain.

In [ ]:
print("Cleaning up downloaded hub operations...")

# Remove only the specific repository we downloaded, not the entire hub cache
if download_result is not None and hasattr(download_result, 'local_path'):
    repo_cache_path = download_result.local_path
    if repo_cache_path.exists():
        # Remove only the specific repository cache
        # The path structure is typically: cache/models--username--repo-name/
        # We want to remove the entire models--username--repo-name directory
        repo_root = repo_cache_path
        # Navigate up to find the repo root (models--username--repo-name)
        while repo_root.parent != repo_root and not repo_root.name.startswith('models--'):
            repo_root = repo_root.parent

        if repo_root.name.startswith('models--'):
            shutil.rmtree(repo_root)
            print(f"✓ Removed specific hub repository cache: {repo_root}")
        else:
            print(f"⚠️ Could not determine repo root from path: {repo_cache_path}")
    else:
        print(f"Hub repository cache path not found: {repo_cache_path}")
else:
    print("⚠️ No download_result available - cannot determine what to clean up")

# Re-import interpretune again
print("\nRe-importing interpretune after cleanup...")

import io
import contextlib

# Capture stdout and stderr during import to check for the expected warning
f_stdout = io.StringIO()
f_stderr = io.StringIO()
with contextlib.redirect_stdout(f_stdout), contextlib.redirect_stderr(f_stderr):
    # Remove interpretune modules again to force reimport and warning emission
    modules_to_remove = [name for name in sys.modules.keys() if name.startswith('interpretune')]
    for module_name in modules_to_remove:
        del sys.modules[module_name]

    # Re-import interpretune
    import interpretune
    from interpretune import DISPATCHER

stdout_output = f_stdout.getvalue()
stderr_output = f_stderr.getvalue()

if stderr_output and "Failed to compile operation 'composite_trivial_test_op'" in stderr_output:
    # note that this warning won't be issued if we've already executed this cell since the latest cache will be used
    print(stderr_output)
    print(
        "Note the above \"Failed to compile operation 'composite_trivial_test_op'\" error on re-import of "
        "interpretune after our cleanup.\n"
        "This is expected: we have removed our hub op definitions (trivial_test_op), but not our local op "
        "definitions (trivial_local_test_op, composite_trivial_test_op).\n"
        "As a result, the locally defined composite operation 'composite_trivial_test_op' could not be "
        "constructed since it depended on the now-missing hub op.\nAll other available operations (local and "
        "built-in) should still be present as we will see."
    )
elif stderr_output:
    print("Unexpected stderr output during import:")
    print(stderr_output)


print("\n ✓ Interpretune re-imported after cleanup")

## Step 8: Verify only local operations remain

Verify that only the local and built-in operations are available after hub cleanup.

In [ ]:
print("Verifying operations after cleanup...")

# Get operation definitions after cleanup
operation_definitions_after = DISPATCHER.registered_ops
total_ops_after = len(operation_definitions_after)

# Get canonical names (resolve aliases to their target operations)
canonical_ops_after = {}
alias_map_after = {}
for op_name, op_def in operation_definitions_after.items():
    canonical_name = op_def.name
    if canonical_name not in canonical_ops_after:
        canonical_ops_after[canonical_name] = op_def
        alias_map_after[canonical_name] = []
    if op_name != canonical_name:
        alias_map_after[canonical_name].append(op_name)

# Count hub operations (those with dots in their canonical names)
hub_ops_after = {name: op_def for name, op_def in canonical_ops_after.items()
                 if '.' in name}

# Count local operations (those without dots but not in the built-in list)
builtin_ops = {'labels_to_ids', 'get_answer_indices', 'get_alive_latents', 'model_forward',
               'model_cache_forward', 'model_ablation', 'model_gradient', 'logit_diffs',
               'logit_diffs_cache', 'sae_correct_acts', 'gradient_attribution', 'ablation_attribution'}

local_ops_after = {name: op_def for name, op_def in canonical_ops_after.items()
                   if '.' not in name and name not in builtin_ops and op_def.composition is None}

# Count composed operations
composed_ops_after = {name: op_def for name, op_def in canonical_ops_after.items()
                      if op_def.composition is not None}

print("\n📊 Operation Summary After Cleanup:")
print(f"  Total registered names: {total_ops_after}")
print(f"  Unique operations: {len(canonical_ops_after)}")
print(f"  Hub operations: {len(hub_ops_after)}")
print(f"  Local operations: {len(local_ops_after)}")
print(f"  Composed operations: {len(composed_ops_after)}")
print(f"  Built-in operations: {len(builtin_ops)}")

if len(hub_ops_after) == 0:
    print("\n✅ Success: No hub operations found - cleanup successful!")
else:
    print(f"\n❌ Warning: {len(hub_ops_after)} hub operations still present:")
    for op_name, op_def in hub_ops_after.items():
        aliases = alias_map_after.get(op_name, [])
        all_names = [op_name] + aliases
        print(f"  - {op_name} (accessible as: {', '.join(all_names)})")

if len(local_ops_after) > 0:
    print("\n🏠 Local operations still available:")
    for op_name, op_def in local_ops_after.items():
        aliases = alias_map_after.get(op_name, [])
        all_names = [op_name] + aliases
        print(f"  - {op_name} (accessible as: {', '.join(all_names)}) - {op_def.description}")
else:
    print("\n⚠️ No local operations found")

# Show remaining operations by category
print("\n📋 Detailed breakdown:")
print(f"\n  Built-in operations ({len(builtin_ops)}):")
for name in sorted(builtin_ops):
    if name in canonical_ops_after:
        aliases = alias_map_after.get(name, [])
        all_names = [name] + aliases
        print(f"    - {name} (accessible as: {', '.join(all_names)})")

if composed_ops_after:
    print(f"\n  Composed operations ({len(composed_ops_after)}):")
    for name in sorted(composed_ops_after.keys()):
        aliases = alias_map_after.get(name, [])
        all_names = [name] + aliases
        print(f"    - {name} (accessible as: {', '.join(all_names)})")

## Cleanup temporary files

Clean up the temporary files created during this example.

In [ ]:
print("Cleaning up temporary files...")

# Remove temporary hub op_collection
if tmp_op_collection.exists():
    shutil.rmtree(tmp_op_collection)
    print(f"✓ Removed temporary hub op_collection: {tmp_op_collection}")

# Remove temporary local op_collection
if tmp_local_op_collection.exists():
    shutil.rmtree(tmp_local_op_collection)
    print(f"✓ Removed temporary local op_collection: {tmp_local_op_collection}")

# Restore the original IT_ANALYSIS_OP_PATHS environment variable
if 'original_op_paths_env' in locals():
    if original_op_paths_env:
        os.environ['IT_ANALYSIS_OP_PATHS'] = original_op_paths_env
        print(f"✓ Restored IT_ANALYSIS_OP_PATHS environment variable to: '{original_op_paths_env}'")
    else:
        if 'IT_ANALYSIS_OP_PATHS' in os.environ:
            del os.environ['IT_ANALYSIS_OP_PATHS']
            print("✓ Unset IT_ANALYSIS_OP_PATHS environment variable")
else:
    print("⚠️ Original IT_ANALYSIS_OP_PATHS value not found, cannot restore")

# Remove from the imported IT_ANALYSIS_OP_PATHS list
if str(tmp_local_op_collection) in IT_ANALYSIS_OP_PATHS:
    IT_ANALYSIS_OP_PATHS.remove(str(tmp_local_op_collection))
    print(f"✓ Removed {tmp_local_op_collection} from imported IT_ANALYSIS_OP_PATHS list")

print(f"\nFinal IT_ANALYSIS_OP_PATHS list: {IT_ANALYSIS_OP_PATHS}")
print(f"Final IT_ANALYSIS_OP_PATHS env var: '{os.environ.get('IT_ANALYSIS_OP_PATHS', '')}'")

print("\n🎉 Hub and Local operations workflow example completed successfully!")
print("\nSummary of what was demonstrated:")
print("1. ✓ Setup local op collection path via IT_ANALYSIS_OP_PATHS environment variable")
print("2. ✓ Copied hub op_collection to /tmp/ with overwrite warning")
print("3. ✓ Uploaded operations to HuggingFace Hub as private repo")
print("4. ✓ Downloaded operations to default hub cache")
print("5. ✓ Re-imported interpretune and verified both hub and local operations")
print("6. ✓ Tested operation instantiation and execution with demo data")
print("7. ✓ Cleaned up hub operations and re-imported")
print("8. ✓ Verified only local and built-in operations remain available")
print("9. ✓ Restored original IT_ANALYSIS_OP_PATHS environment variable")

## Step 9: Final verification after environment cleanup

Re-import interpretune one final time to verify that local operations are no longer available after unsetting IT_ANALYSIS_OP_PATHS.

In [ ]:
print("Final verification: Re-importing interpretune after environment cleanup...")

# Remove interpretune modules from sys.modules to force reimport
modules_to_remove = [name for name in sys.modules.keys() if name.startswith('interpretune')]
for module_name in modules_to_remove:
    del sys.modules[module_name]

# Re-import interpretune one final time
import interpretune
from interpretune import DISPATCHER

print("✓ Interpretune re-imported after environment cleanup")

# Get operation definitions after complete cleanup
operation_definitions_final = DISPATCHER.registered_ops
total_ops_final = len(operation_definitions_final)

# Get canonical names (resolve aliases to their target operations)
canonical_ops_final = {}
alias_map_final = {}
for op_name, op_def in operation_definitions_final.items():
    canonical_name = op_def.name
    if canonical_name not in canonical_ops_final:
        canonical_ops_final[canonical_name] = op_def
        alias_map_final[canonical_name] = []
    if op_name != canonical_name:
        alias_map_final[canonical_name].append(op_name)

# Count operations after complete cleanup
hub_ops_final = {name: op_def for name, op_def in canonical_ops_final.items()
                 if '.' in name}

builtin_ops = {'labels_to_ids', 'get_answer_indices', 'get_alive_latents', 'model_forward',
               'model_cache_forward', 'model_ablation', 'model_gradient', 'logit_diffs',
               'logit_diffs_cache', 'sae_correct_acts', 'gradient_attribution', 'ablation_attribution'}

local_ops_final = {name: op_def for name, op_def in canonical_ops_final.items()
                   if '.' not in name and name not in builtin_ops and op_def.composition is None}

composed_ops_final = {name: op_def for name, op_def in canonical_ops_final.items()
                      if op_def.composition is not None}

print("\n📊 Final Operation Summary (after complete cleanup):")
print(f"  Total registered names: {total_ops_final}")
print(f"  Unique operations: {len(canonical_ops_final)}")
print(f"  Hub operations: {len(hub_ops_final)}")
print(f"  Local operations: {len(local_ops_final)}")
print(f"  Composed operations: {len(composed_ops_final)}")
print(f"  Built-in operations: {len(builtin_ops)}")

# Verify complete cleanup
if len(hub_ops_final) == 0 and len(local_ops_final) == 0:
    print("\n🎯 Perfect! Complete cleanup successful - only built-in and composed operations remain!")
elif len(hub_ops_final) == 0:
    print(f"\n⚠️ Hub operations cleaned up, but {len(local_ops_final)} local operations still present:")
    for op_name, op_def in local_ops_final.items():
        aliases = alias_map_final.get(op_name, [])
        all_names = [op_name] + aliases
        print(f"    - {op_name} (accessible as: {', '.join(all_names)})")
elif len(local_ops_final) == 0:
    print(f"\n⚠️ Local operations cleaned up, but {len(hub_ops_final)} hub operations still present:")
    for op_name, op_def in hub_ops_final.items():
        aliases = alias_map_final.get(op_name, [])
        all_names = [op_name] + aliases
        print(f"    - {op_name} (accessible as: {', '.join(all_names)})")
else:
    print(f"\n❌ Cleanup incomplete: {len(hub_ops_final)} hub ops and {len(local_ops_final)} local ops still present")

print("\nEnvironment verification:")
print(f"  Current IT_ANALYSIS_OP_PATHS env var: '{os.environ.get('IT_ANALYSIS_OP_PATHS', 'Not set')}'")